In [3]:
import pandas as pd
import numpy as np
import os, os.path
import setup_analysis as sa

/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python


/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:287: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:287: UserWarning: Invalid subsector attribute 'key_varreqs_all'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")
/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/data_structures.py:287: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector'
  warnings.warn(f"Invalid subsec

In [4]:
# get the
df_mkw_out = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "tmp_afolu_vars_with_mcmc_new.csv"))


In [5]:
df_mkw_out

,subsector,src,variable,time_series_id,strategy_id,variable_trajectory_group,variable_trajectory_group_trajectory_type,normalize_group,trajgroup_no_vary_q,uniform_scaling_q,...,26,27,28,29,30,31,32,33,34,35
0,Agriculture,from_cr,ef_anaerobicdom_bevs_and_spices_kg_ch4_ha,0.0,0.0,NaN,NaN,NaN,NaN,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Agriculture,from_cr,ef_anaerobicdom_cereals_kg_ch4_ha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Agriculture,from_cr,ef_anaerobicdom_coffee_kg_ch4_ha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Agriculture,from_cr,ef_anaerobicdom_fibers_kg_ch4_ha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Agriculture,from_cr,ef_anaerobicdom_fruits_kg_ch4_ha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,Livestock,from_cr,lvst_pop_initial_goats,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25082.0,25082.0,25082.0,25082.0,25082.0,25082.0,25082.0,25082.0,25082.0,25082.0
306,Livestock,from_cr,lvst_pop_initial_horses,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,126033.0,126033.0,126033.0,126033.0,126033.0,126033.0,126033.0,126033.0,126033.0,126033.0
307,Livestock,from_cr,lvst_pop_initial_mules,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5273.0,5273.0,5273.0,5273.0,5273.0,5273.0,5273.0,5273.0,5273.0,5273.0
308,Livestock,from_cr,lvst_pop_initial_pigs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,462282.0,462282.0,462282.0,462282.0,462282.0,462282.0,462282.0,462282.0,462282.0,462282.0
